In [1]:
Import-Module ImportExcel

In [2]:
$u = Import-Excel './users.xlsx'

In [3]:
$userToIdHash = @{}
$idToUserHash = @{}

$u | ForEach-Object -process {
   $t = @{$_.'Display name' = $_.'Object Id'}
   $userToIdHash += $t

   $t = @{$_.'Object Id' = $_.'Display name'}
   $idToUserHash += $t
}

In [4]:
$u| Select-Object @{Label='Name'; Expression={$_.'Display Name'}}, @{Label='ObjectID'; Expression={$_.'Object Id'}}, @{Label='Manager'; Expression={$_.Manager}},
                  @{Label='Manager ObjectID'; Expression={$userToIdHash[$_.Manager]}}


Name              ObjectID                             Manager         Manager ObjectID
----              --------                             -------         ----------------
Adele Vance       e065d899-0464-4a43-b02a-206028b27982 Patti Fernandez 
Alex Wilber       c3a3bf8e-d9d4-4b9a-b7d4-175993bbe726 Pradeep Gupta   
Daniel Carmack    9a2e7c71-2c59-4b80-ada4-19d8534e6d99 Patti Fernandez 
Diego Siciliani   0a02a0f1-48e1-4010-b904-c7650b34c112 Pradeep Gupta   
Grady Archie      2ea73151-a011-47b5-9a99-8d37adc221b5 Patti Fernandez 
Henrietta Mueller 2c758750-2644-4e13-975f-453e0dfa2263 Pradeep Gupta   
Isaiah Langer     f243a1ce-a5e6-42e1-abe1-f49ca450d6a0 Patti Fernandez 
Johanna Lorenz    40d46eb7-4a18-4fa0-b410-c40c92324624 Pradeep Gupta   
Joni Sherman      96ba2e29-c833-4322-83e1-1f609aa3fe4c Patti Fernandez 
Lee Gu            98d68316-30d8-43c4-ac35-4319ec3b50e9 Pradeep Gupta   
Lidia Holloway    a0df3ed6-b117-4faa-aedf-ab78d0d60ac0 Patti Fernandez 
Lynne Robbins     27baf846-be3a

Error: Command failed: SubmitCode: $u| Select-Object @{Label='Name'; Expression={$_.' ...

## Record tenant user's manager 


In [5]:
Connect-MgGraph

Welcome to Microsoft Graph!

Connected via delegated access using 14d82eec-204b-4c2f-b7e8-296a70dab67e
Readme: https://aka.ms/graph/sdk/powershell
SDK Docs: https://aka.ms/graph/sdk/powershell/docs
API Docs: https://aka.ms/graph/docs

NOTE: You can use the -NoWelcome parameter to suppress this message.



In [6]:
$users = Get-MgUser

In [7]:
$users


DisplayName       Id                                   Mail                              UserPrinci
                                                                                         palName
-----------       --                                   ----                              ----------
Adele Vance       e065d899-0464-4a43-b02a-206028b27982 AdeleV@61b3yr.onmicrosoft.com     AdeleV@61…
Alex Wilber       c3a3bf8e-d9d4-4b9a-b7d4-175993bbe726 AlexW@61b3yr.onmicrosoft.com      AlexW@61b…
Daniel Carmack    9a2e7c71-2c59-4b80-ada4-19d8534e6d99 Daniel.C@61b3yr.onmicrosoft.com   Daniel.C@…
Diego Siciliani   0a02a0f1-48e1-4010-b904-c7650b34c112 DiegoS@61b3yr.onmicrosoft.com     DiegoS@61…
Grady Archie      2ea73151-a011-47b5-9a99-8d37adc221b5 GradyA@61b3yr.onmicrosoft.com     GradyA@61…
Henrietta Mueller 2c758750-2644-4e13-975f-453e0dfa2263 HenriettaM@61b3yr.onmicrosoft.com Henrietta…
Isaiah Langer     f243a1ce-a5e6-42e1-abe1-f49ca450d6a0 IsaiahL@61b3yr.onmicrosoft.com    IsaiahL@6…
Jo

In [23]:
$users | ForEach-Object -Begin {$userManagerHash = @{}} `
                        -Process {
                                    $userManagerId = Get-MgUserManager -UserId $_.Id -ErrorAction SilentlyContinue
                                    if ($userManagerId) {
                                        $userManager = $idToUserHash[$userManagerId.Id]
                                    }
                                    else {
                                        $userManager = 'No Manager'
                                    }
                                    $userManagerHash += @{$_.DisplayName = $userManager}
                                } `
                        -End {$userManagerHash | Format-Table}

                        


Name                           Value
----                           -----
Grady Archie                   Lee Gu
Henrietta Mueller              Lee Gu
Nestor Wilke                   Patti Fernandez
Lynne Robbins                  Miriam Graham
Johanna Lorenz                 Lee Gu
Patti Fernandez                No Manager
Alex Wilber                    Miriam Graham
Daniel Carmack                 No Manager
Miriam Graham                  Patti Fernandez
Isaiah Langer                  Miriam Graham
Pradeep Gupta                  Nestor Wilke
Adele Vance                    Miriam Graham
Joni Sherman                   Nestor Wilke
Megan Bowen                    Miriam Graham
Lee Gu                         Patti Fernandez
Diego Siciliani                Nestor Wilke
P A                            No Manager
Lidia Holloway                 Lee Gu



Error: Command failed: SubmitCode: $users | ForEach-Object -Begin {$userManagerHash = ...